In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import csv
import os
import tensorflow as tf
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
# from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from test_utils import summary, comparator
# import public_tests

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#path='/kaggle/input/aptos2019-blindness-detection/train_images'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def Read_training_data():
    path_training_imgs='/kaggle/input/aptos2019-blindness-detection/train_images'
    path_training_lbls='/kaggle/input/aptos2019-blindness-detection/train.csv'
    train_image_list=np.zeros((3662,256,256,3))
    train_lbls_list=np.zeros((1,3662))
    i=0
    j=0

    with open(path_training_lbls, 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    if(row[1]=='diagnosis'):
                        continue
                    train_lbls_list[0][j]=int(row[1])
                    j=j+1
    Y_train =tf.keras.utils.to_categorical(train_lbls_list, num_classes=5, dtype="float32")
    print(Y_train.shape)
    for dirname, _, filenames in os.walk(path_training_imgs):
        for filename in filenames:
            image_path=os.path.join(dirname, filename)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img,(256,256))
#             img=np.asarray(img)
#             img=img.reshape(224,224,3)
            train_image_list[i]=img
            i=i+1

#     print(training_image_list.shape)
#     for dirname, _, filenames in os.walk(path_training_lbls):
    return train_image_list , Y_train

In [ ]:
X, X_lbls=Read_training_data()
print(X.shape, X_lbls.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D , GaussianDropout
from tensorflow.keras import regularizers, optimizers
k=4
s=2
model = Sequential()
model.add(Conv2D(input_shape=(256,256,3),filters=16,kernel_size=(k,k),padding="valid"))
model.add(Conv2D(filters = 16,kernel_size=(k,k),strides=(s,s),padding = 'valid', activation ='relu', input_shape = (224,224,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(s,s)))
model.add(Conv2D(filters = 64, kernel_size=(k,k),strides=(s,s),padding = 'Same'))
model.add(Conv2D(filters = 64, kernel_size=(k,k),strides=(s,s),padding = 'Same' ))
model.add(Conv2D(filters = 64, kernel_size=(k,k),strides=(s,s),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(s,s)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dense(5, activation = "softmax"))

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999,amsgrad=True,clipnorm=1.,clipvalue=0.5)

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
epochs = 20  # for better result increase the epochs
batch_size = 20

In [ ]:
model.fit(X,X_lbls[0],batch_size=20, epochs=50)

In [ ]:
submit = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/sample_submission.csv')
predicted = []

In [ ]:
from tqdm import tqdm
for i, name in tqdm(enumerate(submit['id_code'])):
    path = os.path.join('/kaggle/input/aptos2019-blindness-detection/test_images/', name+'.png')
    image = cv2.imread(path)
    #print(path)
    image = cv2.resize(image, (256, 256))
    X = np.array((image[np.newaxis])/255)
   # print(X[0][150])
   # print(((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist())
    score_predict=model.predict(X)
    print(score_predict)
    label_predict = np.argmax(score_predict)
    predicted.append(str(label_predict))

In [ ]:
submit['diagnosis'] = predicted
submit.to_csv('submission.csv', index=False)
submit.head()

In [ ]:
print(predicted)